In [1]:
import os
import pandas as pd
import numpy as np
import torch
from torch import nn
from os import sys
sys.path.append('..\..')
from utils import DataLoader

### 1. 读取数据

In [2]:
train_path = os.path.join('.', 'data', 'house-prices-advanced-regression-techniques', 'train.csv')
test_path = os.path.join('.', 'data', 'house-prices-advanced-regression-techniques', 'test.csv')
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)
# train_data
# print(train_data.shape)
# print(test_data.shape)

### 2. 预处理
对数据做预处理，主要包含如下几个步骤：

1. 对特征进行标准化：0均值，单位方差
2. 缺失值处理：填充为相应特征的平均值
3. 处理离散值：可以使用独热编码

In [3]:
all_features = pd.concat([train_data.iloc[:, 1:-1], test_data.iloc[:, 1:]])
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index
all_features[numeric_features] = all_features[numeric_features].apply(lambda x: (x - x.mean()) / (x.std()))
all_features[numeric_features] = all_features[numeric_features].fillna(0)
#type(numeric_features)
#print(all_features.shape)

In [4]:
all_features = pd.get_dummies(all_features, dummy_na=True)

### 3. 将Pandas数据转为NumPy格式，并转换为张量表示

In [5]:
n_train = train_data.shape[0]
train_features = torch.tensor(all_features[:n_train].to_numpy(), dtype=torch.float32)
test_features = torch.tensor(all_features[n_train:].to_numpy(), dtype=torch.float32)
train_labels = torch.tensor(train_data.SalePrice.to_numpy().reshape(-1, 1), dtype=torch.float32)

### 4.训练

In [6]:
loss = nn.MSELoss()
in_features = train_features.shape[1]

def get_net():
    net = nn.Sequential(nn.Linear(in_features, 1))
    return net

""" log RMSE, 对于较大值有压缩效果 """
def log_rmse(net, features, labels):
    clipped_preds = torch.clamp(net(features), 1, float('inf'))
    rmse = torch.sqrt(loss(torch.log(clipped_preds), torch.log(labels)))
    return rmse.item()

def train(net, train_features, train_labels, test_features, test_labels,
          num_epcohs, learning_rate, weight_decay, batch_size):
    train_ls, test_ls = [], []
    train_iter = DataLoader.load_array((train_features, train_labels), batch_size)
    """ using Adam """
    optimizer = torch.optim.Adam(net.parameters(), lr = learning_rate, weight_decay= weight_decay)
    for epoch in num_epcohs:
        for X , y in train_iter:
            optimizer.zero_grad()
            l = loss(net(X), y)
            optimizer.step()